#001

In [1]:
import numpy as np
import pandas as pd
path = u"./dataset/s2t/englishChuanMin.srt"
# path = u"./dataset/001.srt"
x = pd.read_csv(path, sep='\n')
y = np.array(x)

z = []
z.append(np.array(['1']))
for i in y:
    z.append(i)

# for i in z:
#     print(i)

s = ""
idx = 0
subPack = []
pack = []
for i in z:
    subPack.append(i[0])
    idx += 1
    if(idx % 3 == 0):
        pack.append(subPack)
        subPack = []
        idx = 0

start = []
end = []
for i in range(len(pack)):
    s = pack[i][1]
    s = s.replace(" --> ", "")
    s = s[0:8], s[12:20]
    a = list(s)
    pack[i][1] = a


In [2]:

import nltk
import numpy as np
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import to_array

config_path = "./model/uncased_L-12_H-768_A-12/bert_config.json"
checkpoint_path = './model/uncased_L-12_H-768_A-12/bert_model.ckpt'
dict_path = './model/uncased_L-12_H-768_A-12/vocab.txt'

model = build_transformer_model(
    config_path=config_path, checkpoint_path=checkpoint_path, with_nsp=True
)
tokenizer = Tokenizer(dict_path, do_lower_case=True)

In [3]:
pack

[['1', ['00:00:00', '00:00:04'], 'Hey, today is the 1st day of this'],
 ['1',
  ['00:00:04', '00:00:11'],
  "voice. Right? Um, I'm pleased to see you attend course sometime."],
 ['2', ['00:00:12', '00:00:16'], 'This is the 1st time for most of students.'],
 ['3',
  ['00:00:16', '00:00:21'],
  'You probably are not familiar with the detouring that I need.'],
 ['4', ['00:00:22', '00:00:26'], 'Well, uh, it is not a traditional way.'],
 ['5',
  ['00:00:26', '00:00:51'],
  "I-I-I do hope that, um, during our lectureing, there will be a light discussion, and I'm going to raise questions, and you answer or, um, you might expect that, um, teacher through and students through, not traditional way."],
 ['6',
  ['00:00:51', '00:01:02'],
  'So before, before this class, um, I hope that you are ready watch the video as the resources I gave.'],
 ['7', ['00:01:03', '00:01:08'], "So today I'm going to describe the course."],
 ['8', ['00:01:08', '00:01:10'], "So it's, of course, this question."],
 ['9'

In [4]:
token = []
seg = []
token_ids = ""
segment_ids = ""
result = []

var = 0.95

for i in range(len(pack)-1):
    if (len(pack[i][2])+len(pack[i+1][2]) >= 512):
        pack[i][2] = pack[i][2][-128:]

    token_ids, segment_ids = tokenizer.encode(
        pack[i][2], pack[i+1][2])

    token_ids, segment_ids = to_array([token_ids], [segment_ids])

    probas = model.predict([token_ids, segment_ids])[0]
    # print(probas)
    result.append(probas)

for i in range(len(pack)-1):
    pack[i].append(result[i][0])

idxList = []
lessThanHalf = []
lessThanHalf.append('00:00:00')

for i in range(len(pack)-1):
    if(pack[i][3] < var):
        idxList.append(i)
        lessThanHalf.append(pack[i][1][1]) #段落起始時間


In [5]:
pack[0]

['1',
 ['00:00:00', '00:00:04'],
 'Hey, today is the 1st day of this',
 0.048433945]

In [6]:
combineText = []
j = ''
j = pack[0][2] #段落文本
for i in range(len(pack)-1):
    # token_testId, Seg_testId = tokenizer.encode(pack[i][2], pack[i+1][2])
    # token_testId, Seg_testId = to_array([token_testId], [Seg_testId])
    # result = model.predict([token_testId, Seg_testId])[0][0]

    if(pack[i][3] > var):
        j += pack[i+1][2]
        if(i == len(pack)-2):
            j = pack[i+1][2]
            combineText.append(j)
            j = ""
    else:
        j += pack[i+1][2]

        combineText.append(j)
        j = ""


In [7]:
aa = ""
bb = []
for i in range(len(pack)-1):
    # print(pack[i][-1])

    if len(aa) == 0:
        aa = pack[i][2]

    if (pack[i][3] >= var):
        aa += pack[i+1][2]
    else:
        bb.append(aa)
        aa = ""

    if (i == len(pack)-2):
        # print(0)
        if len(aa) != 0:
            bb.append(aa)
        else:
            bb.append(pack[i+1][2])
combineText = bb


In [8]:
combineText

['Hey, today is the 1st day of this',
 "voice. Right? Um, I'm pleased to see you attend course sometime.This is the 1st time for most of students.",
 "You probably are not familiar with the detouring that I need.Well, uh, it is not a traditional way.I-I-I do hope that, um, during our lectureing, there will be a light discussion, and I'm going to raise questions, and you answer or, um, you might expect that, um, teacher through and students through, not traditional way.So before, before this class, um, I hope that you are ready watch the video as the resources I gave.So today I'm going to describe the course.So it's, of course, this question.Here we go. So we have a syllabus.That is the syllabus. That means we could use schedule and the purpose of these schools, the requirements that, um, you should have, and the grading policy.So 1st of all, I'm going to explain that, right?We have a syllabus, right? Um, right?Of course, make sure they come to the right class.I 1st one is system analys

In [9]:
#example
# data = "All work and no play makes jack dull boy. All work and no play makes jack a dull boy."
# stopWords = set(stopwords.words('english'))
# words = word_tokenize(data)
# wordsFiltered=[]
# for w in words:
#     if w not in stopWords:
#         print(w)

In [10]:
for i in range(len(combineText)):
    combineText[i]=combineText[i].lower()
combineText

['hey, today is the 1st day of this',
 "voice. right? um, i'm pleased to see you attend course sometime.this is the 1st time for most of students.",
 "you probably are not familiar with the detouring that i need.well, uh, it is not a traditional way.i-i-i do hope that, um, during our lectureing, there will be a light discussion, and i'm going to raise questions, and you answer or, um, you might expect that, um, teacher through and students through, not traditional way.so before, before this class, um, i hope that you are ready watch the video as the resources i gave.so today i'm going to describe the course.so it's, of course, this question.here we go. so we have a syllabus.that is the syllabus. that means we could use schedule and the purpose of these schools, the requirements that, um, you should have, and the grading policy.so 1st of all, i'm going to explain that, right?we have a syllabus, right? um, right?of course, make sure they come to the right class.i 1st one is system analys

In [11]:
import re
for i in range(len(combineText)):
    combineText[i]=re.sub(r"[^\w\s]","",combineText[i])
    combineText[i]=re.sub(r"[0-9]","",combineText[i])
combineText

['hey today is the st day of this',
 'voice right um im pleased to see you attend course sometimethis is the st time for most of students',
 'you probably are not familiar with the detouring that i needwell uh it is not a traditional wayiii do hope that um during our lectureing there will be a light discussion and im going to raise questions and you answer or um you might expect that um teacher through and students through not traditional wayso before before this class um i hope that you are ready watch the video as the resources i gaveso today im going to describe the courseso its of course this questionhere we go so we have a syllabusthat is the syllabus that means we could use schedule and the purpose of these schools the requirements that um you should have and the grading policyso st of all im going to explain that rightwe have a syllabus right um rightof course make sure they come to the right classi st one is system analysis and design gameand were going to announce any message 

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stopWords=set(stopwords.words("english"))
words_filtered=[]
for word in combineText:
    words=word_tokenize(word)
    for w in words:
        if w not in stopWords:
            words_filtered.append(w)

In [13]:
words_filtered

['hey',
 'today',
 'st',
 'day',
 'voice',
 'right',
 'um',
 'im',
 'pleased',
 'see',
 'attend',
 'course',
 'sometimethis',
 'st',
 'time',
 'students',
 'probably',
 'familiar',
 'detouring',
 'needwell',
 'uh',
 'traditional',
 'wayiii',
 'hope',
 'um',
 'lectureing',
 'light',
 'discussion',
 'im',
 'going',
 'raise',
 'questions',
 'answer',
 'um',
 'might',
 'expect',
 'um',
 'teacher',
 'students',
 'traditional',
 'wayso',
 'class',
 'um',
 'hope',
 'ready',
 'watch',
 'video',
 'resources',
 'gaveso',
 'today',
 'im',
 'going',
 'describe',
 'courseso',
 'course',
 'questionhere',
 'go',
 'syllabusthat',
 'syllabus',
 'means',
 'could',
 'use',
 'schedule',
 'purpose',
 'schools',
 'requirements',
 'um',
 'grading',
 'policyso',
 'st',
 'im',
 'going',
 'explain',
 'rightwe',
 'syllabus',
 'right',
 'um',
 'rightof',
 'course',
 'make',
 'sure',
 'come',
 'right',
 'classi',
 'st',
 'one',
 'system',
 'analysis',
 'design',
 'gameand',
 'going',
 'announce',
 'message',
 'goi

In [14]:
words_count=[]
count={}
for words in words_filtered:
    
    if words in count:
        count[words]+=1
    else:
        count[words]=1
words_count.append(count)


In [15]:
#tf=出現在所有文檔的某個詞 / 全部文檔的詞數(包含重複)
# idf= log(所有文檔 / 某詞出現在某a個文檔) (以10為底)
# tf*idf=tfidf 
keys=list(words_count[0].keys())
tf_freq=[]
for i in range(len(words_count[0])):
    tf_freq.append(words_count[0][keys[i]]/len(words_filtered))

In [16]:
new_tf_freq=words_count[0].copy()
for i in range(len(keys)):
    new_tf_freq[keys[i]]=tf_freq[i]

In [17]:
idf_list={}

for key in keys:
    idf_list[key]=0
    for word in combineText:
        if key in word:
            idf_list[key]+=1


In [18]:
idf_list

{'hey': 11,
 'today': 5,
 'st': 30,
 'day': 9,
 'voice': 1,
 'right': 25,
 'um': 18,
 'im': 21,
 'pleased': 1,
 'see': 6,
 'attend': 1,
 'course': 12,
 'sometimethis': 1,
 'time': 9,
 'students': 6,
 'probably': 9,
 'familiar': 2,
 'detouring': 1,
 'needwell': 1,
 'uh': 6,
 'traditional': 4,
 'wayiii': 1,
 'hope': 9,
 'lectureing': 1,
 'light': 3,
 'discussion': 2,
 'going': 16,
 'raise': 4,
 'questions': 5,
 'answer': 3,
 'might': 4,
 'expect': 5,
 'teacher': 4,
 'wayso': 1,
 'class': 10,
 'ready': 6,
 'watch': 1,
 'video': 7,
 'resources': 3,
 'gaveso': 1,
 'describe': 2,
 'courseso': 1,
 'questionhere': 1,
 'go': 27,
 'syllabusthat': 1,
 'syllabus': 1,
 'means': 2,
 'could': 3,
 'use': 18,
 'schedule': 3,
 'purpose': 2,
 'schools': 2,
 'requirements': 3,
 'grading': 1,
 'policyso': 1,
 'explain': 2,
 'rightwe': 1,
 'rightof': 2,
 'make': 6,
 'sure': 6,
 'come': 11,
 'classi': 2,
 'one': 17,
 'system': 5,
 'analysis': 2,
 'design': 5,
 'gameand': 1,
 'announce': 3,
 'message': 1,
 'u

In [19]:
keys=list(words_count[0].keys())
inverse_document_freqency=[]
for i in range(len(idf_list)):
    inverse_document_freqency.append(len(combineText)/idf_list[keys[i]])

In [20]:
keys=list(words_count[0].keys())
tfidf=np.array(tf_freq)*np.array(inverse_document_freqency)
tfidf_dict={}

for i in range(len(keys)):
    tfidf_dict[keys[i]]=tfidf[i]

In [21]:
tfidf_dict

{'hey': 0.0028278173439463764,
 'today': 0.03732718894009217,
 'st': 0.005184331797235023,
 'day': 0.01728110599078341,
 'voice': 0.03110599078341014,
 'right': 0.0447926267281106,
 'um': 0.06912442396313365,
 'im': 0.02221856484529296,
 'pleased': 0.03110599078341014,
 'see': 0.025921658986175117,
 'attend': 0.03110599078341014,
 'course': 0.0699884792626728,
 'sometimethis': 0.03110599078341014,
 'time': 0.020737327188940093,
 'students': 0.03110599078341014,
 'probably': 0.03110599078341014,
 'familiar': 0.03110599078341014,
 'detouring': 0.03110599078341014,
 'needwell': 0.03110599078341014,
 'uh': 0.041474654377880185,
 'traditional': 0.03110599078341014,
 'wayiii': 0.03110599078341014,
 'hope': 0.04838709677419355,
 'lectureing': 0.03110599078341014,
 'light': 0.010368663594470046,
 'discussion': 0.03110599078341014,
 'going': 0.0699884792626728,
 'raise': 0.01555299539170507,
 'questions': 0.031105990783410142,
 'answer': 0.03110599078341014,
 'might': 0.03110599078341014,
 'exp

In [22]:
tfidfResult={k: v for k, v in sorted(tfidf_dict.items(), key=lambda item: item[1],reverse=True)}

In [23]:
Top20TfResult=[]
for i in range(20):
    Top20TfResult.append(list(tfidfResult.items())[i][0])

In [24]:
wst = [] #分詞後文本
for sentence in combineText:
    seq_list = word_tokenize(sentence) 
    wst.append(seq_list)

In [25]:
wst

[['hey', 'today', 'is', 'the', 'st', 'day', 'of', 'this'],
 ['voice',
  'right',
  'um',
  'im',
  'pleased',
  'to',
  'see',
  'you',
  'attend',
  'course',
  'sometimethis',
  'is',
  'the',
  'st',
  'time',
  'for',
  'most',
  'of',
  'students'],
 ['you',
  'probably',
  'are',
  'not',
  'familiar',
  'with',
  'the',
  'detouring',
  'that',
  'i',
  'needwell',
  'uh',
  'it',
  'is',
  'not',
  'a',
  'traditional',
  'wayiii',
  'do',
  'hope',
  'that',
  'um',
  'during',
  'our',
  'lectureing',
  'there',
  'will',
  'be',
  'a',
  'light',
  'discussion',
  'and',
  'im',
  'going',
  'to',
  'raise',
  'questions',
  'and',
  'you',
  'answer',
  'or',
  'um',
  'you',
  'might',
  'expect',
  'that',
  'um',
  'teacher',
  'through',
  'and',
  'students',
  'through',
  'not',
  'traditional',
  'wayso',
  'before',
  'before',
  'this',
  'class',
  'um',
  'i',
  'hope',
  'that',
  'you',
  'are',
  'ready',
  'watch',
  'the',
  'video',
  'as',
  'the',
  'res

In [26]:
TF = np.zeros((np.shape(wst)[0], 20)) #產生0陣列 np.shape(TF) = (len(wst), 20)

combineStr = "".join(combineText)

#取詞頻前20的名詞 TFIDF 

Query = Top20TfResult

for i in range(len(Query)): #Query循環
    for j in range(len(wst)): #wst循環
        for k in range(len(wst[j])): #找出wst[j]中出現了那些top20keyWord
            if(Query[i] == wst[j][k]):
                TF[j][i] = 1
                continue
# print(TF)
keyWord = []
deliver = []
for i in TF:
    for j in range(len(i)):
        if(i[j]):
            deliver.append(Query[j])
    keyWord.append(deliver)
    deliver = []

finalResult = []
tmpProcess = []
for i in range(len(combineText)):
    tmpProcess.append(lessThanHalf[i])
    # tmpProcess.append(lessThanHalf[i+1])
    tmpProcess.append(combineText[i])
    tmpProcess.append(keyWord[i])
    finalResult.append(tmpProcess)
    tmpProcess = []


In [27]:

import numpy as np

In [28]:
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import to_array



model = build_transformer_model(
    config_path, 
    checkpoint_path
)
tokenizer = Tokenizer(dict_path, do_lower_case=True)

In [29]:
def Euclidean_Distance(v1, v2): #歐式距離
    return np.sqrt(np.sum((v1-v2)**2))


In [30]:
SS=""

for i in range(len(finalResult)):
    SS=SS.join(finalResult[i][2]) #出現的top20keyWord
    print(SS)
    token_ids, segment_ids = tokenizer.encode(SS)
    token_ids, segment_ids = to_array([token_ids], [segment_ids])  
    finalResult[i].append(model.predict([token_ids, segment_ids])[0][0])
    #print(model.predict([token_ids, segment_ids])[0][0])
    SS=""


courseum
coursegoingumsyllabusknow
theorycoursegoingumknowpractice
englishlisteningcoursegoingumdontknowpracticeoftenspeakingenvironmentssituationglobaleventually
dont




um
dont
um

course

goingumdontknow
umknow


teamworkparticipationgoingumknow
personalteamworkparticipationcoursegoingumdontknow

coursegoingumknow
know

participation
englishgoingum
course

versioncoursegoingumknow

versiongoingumdontknow



um
dont
version
coursegoingum
goingdont
know
going
goingknow

going


going


coursedont




In [31]:
import math
dist_result=[]
for i in range(len(finalResult)-1):
    finalResult[i].append(Euclidean_Distance(finalResult[i][3],finalResult[i+1][3]))

In [32]:
#dist_result=dist_result[:-1]#沒東西可比 拿掉最後一個
for i in range(len(finalResult)-1):
    dist_result.append(finalResult[i][-1])

In [33]:
trueFinalResult = []
trueText = ""
tureTime = ''
trueKW = []
first = 0
for i in range(len(finalResult)-1):
    #print(trueFinalResult)
    if len(trueText) == 0:
        tureTime = finalResult[i][0]
        trueText = finalResult[i][1]
        trueKW = finalResult[i][2]

    if (finalResult[i][-1] <= 10 or len(finalResult[i][2])==0):
        trueText += finalResult[i+1][1]
        trueKW = list(set(trueKW).union(set(finalResult[i+1][2])))
        #print(tureTime, trueText, trueKW, first)
    else:
        trueFinalResult.append([tureTime,
                                trueText,
                                trueKW
                                ])
        trueText = ""
        tureTime = ''
        trueKW = []
        #print(tureTime, trueText, trueKW, first)
    
    if i==len(dist_result)-1:
        if len(trueText) != 0:
            trueFinalResult.append([tureTime,
                                trueText,
                                trueKW
                                ])
                #print(tureTime, trueText, trueKW, first)
        else:
            print(4)
            tureTime = finalResult[i+1][0]
            trueText = finalResult[i+1][1]
            trueKW = finalResult[i+1][2]
            trueFinalResult.append([tureTime,
                                trueText,
                                trueKW
                                ])
            trueText = ""
            tureTime = ''
            trueKW = []
                #print(tureTime, trueText, trueKW, first)
    
    
# fi    rst=dist_result[i]
# se    cond=dist_result[i+1]
# if    (first<second and first*1.25>second):
#         trueText+=finalResult[i][1]+finalResult[i+1][1]
#         trueFinalResult.append(finalResult[i][0])
#         trueFinalResult.append(trueText)
    

In [34]:
list(set(finalResult[0][2]).union(set(finalResult[1][2])))

['course', 'um']

In [35]:
len(trueFinalResult)

28

In [36]:
trueFinalResult

[['00:00:00',
  'hey today is the st day of thisvoice right um im pleased to see you attend course sometimethis is the st time for most of students',
  ['course', 'um']],
 ['00:00:16',
  'you probably are not familiar with the detouring that i needwell uh it is not a traditional wayiii do hope that um during our lectureing there will be a light discussion and im going to raise questions and you answer or um you might expect that um teacher through and students through not traditional wayso before before this class um i hope that you are ready watch the video as the resources i gaveso today im going to describe the courseso its of course this questionhere we go so we have a syllabusthat is the syllabus that means we could use schedule and the purpose of these schools the requirements that um you should have and the grading policyso st of all im going to explain that rightwe have a syllabus right um rightof course make sure they come to the right classi st one is system analysis and desi

In [ ]:
[ ['00:00:00','course', 'um']]

In [45]:
spl='20:40:25'.split(":")

In [46]:
spl

['20', '40', '25']

In [50]:
str(int(spl[0])*60+int(spl[1]))+":"+spl[2]

'1240:25'

In [54]:
path="C:/Users/Selab-001/Documents/nlp/Output.txt"

In [62]:
f=open(path,"w")
f.writelines([ '00:00:00',' course,', 'um'])
f.close()

In [66]:
trueFinalResult[0][0]

'00:00:00'

In [76]:
trueFinalResult[0][2]

['course', 'um']

In [121]:
temp=[]
temp.append(trueFinalResult[0][0])

In [122]:
strk=[" "+trueFinalResult[0][2][0]+",",trueFinalResult[0][2][1]]
strk

[' course,', 'um']

In [123]:
temp.extend(strk)

In [124]:
temp

['00:00:00', ' course,', 'um']

In [155]:
f=open(path,"w")
for i in range(len(trueFinalResult)):
    spl=trueFinalResult[i][0].split(":")
    tmp=str(int(spl[0])*60+int(spl[1]))+":"+spl[2]

    spl=tmp.split(":")
    if int(spl[0])<10:
        tmp="0"+tmp
    f.writelines(tmp)
    strk=' '
    for i in trueFinalResult[i][2]:
        strk=strk+i+','
    f.writelines(strk+"\n")
f.close()

In [139]:
spl

['00', '40', '25']

In [149]:
spl="00:25"
spl=spl.split(":")

tmp=str(int(spl[0])*60+int(spl[1]))+":"+spl[2]
spl=tmp.split(":")

IndexError: list index out of range

In [151]:
spl=['08', '25']

In [152]:
if int(spl[0])<10:
    tmp="0"+tmp

In [153]:
tmp

'040:25'